- import lib and check version

In [1]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import seaborn as sns
import math
import matplotlib.pyplot as plt
import numpy as np
import re
import demoji
import datetime

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

/Users/frostace/.conda/envs/bert_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/frostace/.conda/envs/bert_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/frostace/.conda/envs/bert_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/frostace/.conda/envs/bert_env/lib

Tensor Flow Version: 1.13.1
Keras Version: 2.2.4-tf

Python 3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 15:01:53) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Pandas 1.0.1
Scikit-Learn 0.22.1
GPU is NOT AVAILABLE


- init demoji

In [2]:
demoji.download_codes()

... OK (Got response in 0.12 seconds)
Writing emoji data to /Users/frostace/.demoji/codes.json ...
... OK


- start server service in terminal

In [3]:
# source activate bert_env
# bert-serving-start -model_dir ./model/tmp/english_L-12_H-768_A-12/ -num_worker=1

- start client service

In [4]:
from bert_serving.client import BertClient
bc = BertClient()
# bc.encode(['First do it', 'then do it right', 'then do it better'])

- data cleaning function

In [5]:
# japanese unicode ranges
ranges = [
  {"from": ord(u"\u3300"), "to": ord(u"\u33ff")},         # compatibility ideographs
  {"from": ord(u"\ufe30"), "to": ord(u"\ufe4f")},         # compatibility ideographs
  {"from": ord(u"\uf900"), "to": ord(u"\ufaff")},         # compatibility ideographs
  {"from": ord(u"\U0002F800"), "to": ord(u"\U0002fa1f")}, # compatibility ideographs
  {'from': ord(u'\u3040'), 'to': ord(u'\u309f')},         # Japanese Hiragana
  {"from": ord(u"\u30a0"), "to": ord(u"\u30ff")},         # Japanese Katakana
  {"from": ord(u"\u2e80"), "to": ord(u"\u2eff")},         # cjk radicals supplement
  {"from": ord(u"\u4e00"), "to": ord(u"\u9fff")},
  {"from": ord(u"\u3400"), "to": ord(u"\u4dbf")},
  {"from": ord(u"\U00020000"), "to": ord(u"\U0002a6df")},
  {"from": ord(u"\U0002a700"), "to": ord(u"\U0002b73f")},
  {"from": ord(u"\U0002b740"), "to": ord(u"\U0002b81f")},
  {"from": ord(u"\U0002b820"), "to": ord(u"\U0002ceaf")}  # included as of Unicode 8.0
]
    
def is_cjk(char):
        return any([range["from"] <= ord(char) <= range["to"] for range in ranges])
    
def data_cleaning(date_str):
    csv_file = pd.read_csv("./data_tweets/tweets-100-" + date_str +".csv", lineterminator='\n')
    raw_tweet = csv_file.tweet.to_list()
    
    rows_to_drop = []
    for i in range(len(raw_tweet)):
        line = raw_tweet[i]
        # remove japanese
        containJap = False
        for ch in line:
            if is_cjk(ch): 
                rows_to_drop.append(i)
                containJap = True
                break
        if containJap: continue

        # remove emoji
        # emoji_pattern = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
        # line = emoji_pattern.sub(r'', line)
        line = demoji.replace(line)

        # remove url
        line = re.sub(r"http\S+", "", line)
        # TODO: remove url ended with .com

    csv_file['tweet'] = raw_tweet
    csv_file = csv_file.drop(rows_to_drop, axis=0)
    
    return csv_file['tweet'].to_list()

- convert tweet to vector

In [6]:
# Inclusive on both sides
start_date = datetime.date(2019, 2, 6)
end_date = datetime.date(2020, 10, 15)
time_window = (end_date - start_date).days + 1

for single_date in (start_date + datetime.timedelta(n) for n in range(time_window)): 
    this_date = single_date.strftime("%Y-%m-%d")
    tweet_list = data_cleaning(this_date)
    tweet_vector_matrix = bc.encode(tweet_list)
    tweet_vector_df = pd.DataFrame(tweet_vector_matrix)
    tweet_vector_df.to_csv("./data_vectors/vectors-100-" + this_date + ".csv", index=False)

/Users/frostace/.conda/envs/bert_env/lib/python3.6/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


FileNotFoundError: [Errno 2] File ./data_tweets/tweets-100-2019-10-16.csv does not exist: './data_tweets/tweets-100-2019-10-16.csv'